## Registration Service demo
compatible with EDC v0.10.1

In [1]:
import requests
import json

from dataspace_apis import *

### Demo setup

In [6]:

IS_LOCALHOST_DEPLOYMENT = False

# for paas:
PROVIDER_URL = "https://provider-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_URL = "https://consumer-edc-connector.apps.paas-dev.psnc.pl"
REGISTRATION_SERVICE_URL = "https://registration-service-backend-edc-connector.apps.paas-dev.psnc.pl" # not deployed yet
FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.paas-dev.psnc.pl"

## for local:
LOCALHOST = "http://localhost"
CONSUMER_CONTAINER = "http://consumer-connector"
PROVIDER_CONTAINER = "http://provider-connector"
REGISTRATION_SERVICE_CONTAINER = "http://registration-service-backend"
FEDERATED_CATALOG_BASE_CONTAINER = "http://federated-catalog"

if (IS_LOCALHOST_DEPLOYMENT):
    CONSUMER_URL = LOCALHOST
    PROVIDER_URL = LOCALHOST
    REGISTRATION_SERVICE_URL = LOCALHOST
    FEDERATED_CATALOG_BASE_URL = LOCALHOST

In [7]:
PROVIDER_API = f"{PROVIDER_URL}/api"
PROVIDER_CONTROL = f"{PROVIDER_URL}/control"
PROVIDER_MANAGEMENT = f"{PROVIDER_URL}/management"
PROVIDER_PROTOCOL = f"{PROVIDER_URL}/protocol"
PROVIDER_PUBLIC = f"{PROVIDER_URL}/public"

CONSUMER_API = f"{CONSUMER_URL}/api"
CONSUMER_CONTROL = f"{CONSUMER_URL}/control"
CONSUMER_MANAGEMENT = f"{CONSUMER_URL}/management"
CONSUMER_PROTOCOL = f"{CONSUMER_URL}/protocol"
CONSUMER_PUBLIC = f"{CONSUMER_URL}/public"

AUTHORITY_URL = f"{REGISTRATION_SERVICE_URL}/authority"
PARTICIPANTS_ENDPOINT_URL = f"{AUTHORITY_URL}/registry/participants"

FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}/catalog"

if (IS_LOCALHOST_DEPLOYMENT):
    PROVIDER_API = f"{PROVIDER_URL}:19191/api"
    PROVIDER_CONTROL = f"{PROVIDER_URL}:19192/control"
    PROVIDER_MANAGEMENT = f"{PROVIDER_URL}:19193/management"
    PROVIDER_PROTOCOL = f"{PROVIDER_URL}:19194/protocol"
    PROVIDER_PUBLIC = f"{PROVIDER_URL}:19291/public"

    CONSUMER_API = f"{CONSUMER_URL}:29191/api"
    CONSUMER_CONTROL = f"{CONSUMER_URL}:29192/control"
    CONSUMER_MANAGEMENT = f"{CONSUMER_URL}:29193/management"
    CONSUMER_PROTOCOL = f"{CONSUMER_URL}:29194/protocol"
    CONSUMER_PUBLIC = f"{CONSUMER_URL}:29291/public"

    AUTHORITY_URL = f"{REGISTRATION_SERVICE_URL}:38182/authority"
    PARTICIPANTS_ENDPOINT_URL = f"{AUTHORITY_URL}/registry/participants"

    FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}:9181/catalog"

In [8]:
provider_control_internal = PROVIDER_CONTROL.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_public_internal = PROVIDER_PUBLIC.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_protocol_internal = f"{PROVIDER_PROTOCOL}".replace(LOCALHOST, PROVIDER_CONTAINER)

default_headers = {
    "Content-Type": "application/json"
}

# participant for tests (add/update/delete)
did = "consumer"
protocolUrl = "http://consumer-connector:29194/protocol"

## Registration Service main operations

### Show dataspace participants

In [9]:
def show_participants():
    return requests.get(PARTICIPANTS_ENDPOINT_URL).json()

participants = show_participants()
print("Stored participants in the Registration Service database:")
print(participants)

Stored participants in the Registration Service database:
[{'did': 'provider', 'status': 'ONBOARDED', 'protocolUrl': 'https://provider-edc-connector.apps.paas-dev.psnc.pl/protocol', 'claims': {'purpose': 'marketing', 'region': 'pl'}}, {'did': 'consumer', 'status': 'ONBOARDED', 'protocolUrl': 'https://consumer-edc-connector.apps.paas-dev.psnc.pl/protocol', 'claims': {'purpose': 'commercial', 'region': 'eu'}}]


### Add a new participant

In [9]:
def add_participant(did, protocolUrl):
    requests.post(f"{PARTICIPANTS_ENDPOINT_URL}?did={did}&protocolUrl={protocolUrl}")

add_participant(did, protocolUrl)

In [10]:
show_participants()

[{'did': 'provider',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://provider-connector:19194/protocol',
  'claims': {'region': 'pl'}},
 {'did': 'consumer',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://consumer-connector:29194/protocol',
  'claims': {}}]

In [11]:
fetch_catalog(FEDERATED_CATALOG_URL).json()

[{'@id': 'dfcca92d-035b-46ab-8037-101525b9f2f8',
  '@type': 'dcat:Catalog',
  'dcat:dataset': {'@id': 'consumer-asset',
   '@type': 'dcat:Dataset',
   'odrl:hasPolicy': {'@id': 'Y29uc3VtZXItcG9sLWNk:Y29uc3VtZXItYXNzZXQ=:ZTI3NmZkNjEtMGIyMi00NGRmLTkyYWQtZWFmOWUyYWNmYjAy',
    '@type': 'odrl:Offer',
    'odrl:permission': [],
    'odrl:prohibition': [],
    'odrl:obligation': []},
   'dcat:distribution': [{'@type': 'dcat:Distribution',
     'dct:format': {'@id': 'HttpData-PULL'},
     'dcat:accessService': {'@id': '38129889-7148-42ab-adfb-99b59aa81942',
      '@type': 'dcat:DataService'}},
    {'@type': 'dcat:Distribution',
     'dct:format': {'@id': 'HttpData-PUSH'},
     'dcat:accessService': {'@id': '38129889-7148-42ab-adfb-99b59aa81942',
      '@type': 'dcat:DataService'}}],
   'proxyPath': 'true',
   'version': '',
   'name': 'consumer-asset',
   'proxyQueryParams': 'true',
   'id': 'consumer-asset',
   'contenttype': 'application/json',
   'baseUrl': 'http://data-source:5000/data'},

### Update participant's status

In [12]:
"""
Available statuses:
---
ONBOARDING_INITIATED(0), // onboarding request received
AUTHORIZING(100), // verifying participants credentials
AUTHORIZED(200), // participant is authorized
ONBOARDED(300), // participant is fully onboarded
DENIED(400), // participant onboarding request denied
FAILED(-1), // participant onboarding failed
DELETED(-100);
"""

def update_participant_status(did, newStatus):
    requests.patch(f"{PARTICIPANTS_ENDPOINT_URL}/{did}?status={newStatus}")

update_participant_status(did, "DENIED")

In [14]:
update_participant_status(did, "ONBOARDED")

In [15]:
show_participants()

[{'did': 'provider',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://provider-connector:19194/protocol',
  'claims': {'region': 'pl'}},
 {'did': 'consumer',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://consumer-connector:29194/protocol',
  'claims': {}}]

### Update participant's claims

In [16]:
def update_claims(did, claims):
    requests.patch(
        headers=default_headers,
        data=json.dumps(claims),
        url=f"{PARTICIPANTS_ENDPOINT_URL}/{did}/claims")

update_claims(did, { "region": "eu" })

In [18]:
update_claims(did, { "region": "pl" })

In [19]:
show_participants()

[{'did': 'provider',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://provider-connector:19194/protocol',
  'claims': {'region': 'pl'}},
 {'did': 'consumer',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://consumer-connector:29194/protocol',
  'claims': {'region': 'pl'}}]

## Perform asset transfer

### Create example asset, policy and policy definition

In [20]:
asset_id = "registration-service-notebook-example-asset"

asset = create_asset(asset_id, PROVIDER_MANAGEMENT, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
asset.status_code

409

In [21]:
policy_id = "registration-service-notebook-example-policy"
allowed_policy_region = "pl"

allowed_region_rule = {
    "action": "use", 
    "constraint": { 
        "@type": "AtomicConstraint", 
        "leftOperand": "https://w3id.org/edc/v0.0.1/ns/regionLocation", 
        "operator": "odrl:eq", 
        "rightOperand": allowed_policy_region 
    }
}

policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[allowed_region_rule])

# expected 200 or 409 (already exists) because the @id is fixed in the example
print(policy.status_code)

409


In [22]:
contract_definition_id = "registration-service-notebook-example-contract-definition"

contract_definition = create_contract_definition(contract_definition_id, PROVIDER_MANAGEMENT, asset_id, policy_id, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
contract_definition.status_code

409

### Fetch catalog, negotiate and perform transfer

In [28]:
import time

def get_offer_id(fetched_catalog, asset_id):

    catalogs_array = []
    if isinstance(fetched_catalog, list):
        catalogs_array = fetched_catalog
    else:
        catalogs_array = [fetched_catalog]

    offer_id = None
    for catalog in catalogs_array:
        dcat_dataset = catalog["dcat:dataset"]
        dataset_array = []
        if isinstance(dcat_dataset, list):
            dataset_array = dcat_dataset
        else:
            dataset_array = [dcat_dataset]

        for asset in dataset_array:
            if (asset["@id"] == asset_id):
                policy = asset["odrl:hasPolicy"]
                if isinstance(policy, list):
                    return policy[0]["@id"]
                else:
                    return policy["@id"]
    return offer_id

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL)
print(f"1. Catalog fetch: {fetched_catalog.status_code}")

offer_id = get_offer_id(fetched_catalog.json(), asset_id)
print(offer_id)

# 2. negotiate a contract
negotiated_contract = negotiate_contract(
    offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)
print(f"2. Negotiate a contract: {negotiated_contract.status_code}")

# wait until the negotiation will be finalized
time.sleep(5)

contract_negotiation_id = negotiated_contract.json()["@id"]
contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
print(contract_agreement.json())

contract_agreement_id = ""
if ("contractAgreementId" in contract_agreement.json()):
    contract_agreement_id = contract_agreement.json()["contractAgreementId"]
else:
    contract_agreement_id = contract_agreement.json()["@id"]

print(contract_agreement_id)

# 3. transfer asset
requested_transfer = request_consumer_pull_transfer(
    "provider",
    CONSUMER_MANAGEMENT,
    "http://consumer-backend:4000/edr-endpoint",
    provider_protocol_internal,
    contract_agreement_id,
    default_headers
)
print(requested_transfer.json())
print(f"3. Transfer an asset: {requested_transfer.status_code}")

1. Catalog fetch: 200
cmVnaXN0cmF0aW9uLXNlcnZpY2Utbm90ZWJvb2stZXhhbXBsZS1jb250cmFjdC1kZWZpbml0aW9u:cmVnaXN0cmF0aW9uLXNlcnZpY2Utbm90ZWJvb2stZXhhbXBsZS1hc3NldA==:NzQwMzc3NTUtZTkwZS00ODc1LWEwNmYtMjM0ZDBiZDg2MThi
2. Negotiate a contract: 200
{'@type': 'ContractNegotiation', '@id': '3c599253-e19b-4801-a2bc-c1f14574fef4', 'type': 'CONSUMER', 'protocol': 'dataspace-protocol-http', 'state': 'FINALIZED', 'counterPartyId': 'provider', 'counterPartyAddress': 'http://provider-connector:19194/protocol', 'callbackAddresses': [], 'createdAt': 1737629167616, 'contractAgreementId': '274e2d81-ef75-48b9-898f-30d7323485c8', '@context': {'@vocab': 'https://w3id.org/edc/v0.0.1/ns/', 'edc': 'https://w3id.org/edc/v0.0.1/ns/', 'odrl': 'http://www.w3.org/ns/odrl/2/'}}
274e2d81-ef75-48b9-898f-30d7323485c8
{'@type': 'IdResponse', '@id': 'c2a9c3f5-ab29-46ef-a18e-57efc304b733', 'createdAt': 1737629172648, '@context': {'@vocab': 'https://w3id.org/edc/v0.0.1/ns/', 'edc': 'https://w3id.org/edc/v0.0.1/ns/', 'odrl': 'ht

In [25]:
transfer_id = requested_transfer.json()["@id"]

get_transfer_state(CONSUMER_MANAGEMENT, transfer_id)

{'@type': 'TransferState',
 'state': 'STARTED',
 '@context': {'@vocab': 'https://w3id.org/edc/v0.0.1/ns/',
  'edc': 'https://w3id.org/edc/v0.0.1/ns/',
  'odrl': 'http://www.w3.org/ns/odrl/2/'}}

### Remove a participant from the dataspace

In [6]:
def remove_participant(did):
    requests.delete(f"{PARTICIPANTS_ENDPOINT_URL}/{did}")

remove_participant(did)

In [26]:
show_participants()

[{'did': 'provider',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://provider-connector:19194/protocol',
  'claims': {'region': 'pl'}},
 {'did': 'consumer',
  'status': 'ONBOARDED',
  'protocolUrl': 'http://consumer-connector:29194/protocol',
  'claims': {'region': 'pl'}}]

In [27]:
fetch_catalog(FEDERATED_CATALOG_URL).json()

[{'@id': 'c0e37fa0-6c34-4e12-b401-8ad458febabc',
  '@type': 'dcat:Catalog',
  'dcat:dataset': [{'@id': 'asset-workshop-demo-by-api',
    '@type': 'dcat:Dataset',
    'odrl:hasPolicy': [{'@id': 'dGVzdC1jZC0wMDE=:YXNzZXQtd29ya3Nob3AtZGVtby1ieS1hcGk=:ODBlNDE1MGUtYjk4NC00M2Y2LWI0YWItNDlhNWNjNDUzYjUz',
      '@type': 'odrl:Offer',
      'odrl:permission': [],
      'odrl:prohibition': [],
      'odrl:obligation': []},
     {'@id': 'ZGVmYXVsdA==:YXNzZXQtd29ya3Nob3AtZGVtby1ieS1hcGk=:OTcxODNkMmYtYzNjMC00ZGNlLWFhYTQtZGU4OTRjNTg4OTVl',
      '@type': 'odrl:Offer',
      'odrl:permission': [],
      'odrl:prohibition': [],
      'odrl:obligation': []}],
    'dcat:distribution': [{'@type': 'dcat:Distribution',
      'dct:format': {'@id': 'HttpData-PULL'},
      'dcat:accessService': {'@id': '4457cc87-fe2b-4af1-b5b7-b959a12a4661',
       '@type': 'dcat:DataService'}},
     {'@type': 'dcat:Distribution',
      'dct:format': {'@id': 'HttpData-PUSH'},
      'dcat:accessService': {'@id': '4457cc87-fe2b